# Tissue Expression Filter Test with TCGA LAML Data

Testing tissue expression functionality with real TCGA Acute Myeloid Leukemia data.


In [3]:
import sys
import os
import pandas as pd
from IPython.display import display

# Add src to path
sys.path.insert(0, os.path.join('..', '..', '..', 'src'))

from pyMut.input import read_maf
from pyMut.filters.tissue_expression import tissue_expression


## Load TCGA LAML Dataset


In [4]:
# Load real TCGA LAML data
maf_path = os.path.join('..', '..', '..', 'src', 'pyMut', 'data', 'examples', 'MAF','tcga_laml.maf.gz')
# TCGA data is typically based on GRCh37 assembly
py_mut = read_maf(maf_path, assembly="37")

print(f"Loaded TCGA LAML data: {len(py_mut.data)} variants")
print(f"Unique genes: {py_mut.data['Hugo_Symbol'].nunique()}")
print(f"Unique samples: {py_mut.data['Tumor_Sample_Barcode'].nunique()}")


2025-07-30 23:17:51,159 | INFO | pyMut.input | Starting MAF reading: ../../../src/pyMut/data/examples/MAF/tcga_laml.maf.gz
2025-07-30 23:17:51,160 | INFO | pyMut.input | Loading from cache: ../../../src/pyMut/data/examples/MAF/.pymut_cache/tcga_laml.maf_3515f757055e6890.parquet
2025-07-30 23:17:51,186 | INFO | pyMut.input | Cache loaded successfully in 0.03 seconds


Loaded TCGA LAML data: 2207 variants
Unique genes: 1611
Unique samples: 193


## Test 1: Individual Gene Expression (with prints)


In [5]:
# Test individual gene expression with genes from the dataset
sample_genes = py_mut.data['Hugo_Symbol'].value_counts().head(3).index.tolist()
print(f"Testing genes: {sample_genes}")

for gene in sample_genes:
    result_laml = tissue_expression(gene, ["LAML", 5])  # LAML = Acute Myeloid Leukemia
    result_blca = tissue_expression(gene, ["BLCA", 5])  # BLCA = Bladder Cancer
    print(f"{gene} - LAML (>5): {result_laml}, BLCA (>5): {result_blca}")


Testing genes: ['DNMT3A', 'FLT3', 'NPM1']
DNMT3A - LAML (>5): False, BLCA (>5): False
FLT3 - LAML (>5): False, BLCA (>5): False
NPM1 - LAML (>5): False, BLCA (>5): True


## Test 2: PyMutation Object Filtering


In [6]:
print(f"Original TCGA data: {len(py_mut.data)} variants")

# Filter by LAML tissue (should keep many since this is LAML data)
filtered_laml = py_mut.filter_by_tissue_expression([('LAML', 5)])
print(f"LAML expressed (>5): {len(filtered_laml.data)} variants")


# Filter by multiple tissues
filtered_multi = py_mut.filter_by_tissue_expression([('LAML', 5), ('BRCA', 3), ('LUAD', 4)])
print(f"Multi-tissue expressed: {len(filtered_multi.data)} variants")


# Filter for NOT expressed in LAML with high threshold
filtered_not = py_mut.filter_by_tissue_expression([('LAML', 50)], keep_expressed=False)
print(f"NOT highly expressed in LAML: {len(filtered_not.data)} variants")


Original TCGA data: 2207 variants
LAML expressed (>5): 0 variants
Multi-tissue expressed: 1010 variants
NOT highly expressed in LAML: 2207 variants


## Test 3: Tissue Expression Results Analysis


In [7]:
# Display tissue expression results dataframes
print("📊 Detailed tissue expression analysis results:")

if hasattr(filtered_multi, 'tissue_expression_results'):
    results_df = filtered_multi.tissue_expression_results
    print(f"\nResults dataframe shape: {results_df.shape}")
    print("Columns:", list(results_df.columns))

    print("\n📊 Complete tissue expression results:")
    display(results_df)

    # Create summary statistics table
    summary_stats = []
    for col in results_df.columns:
        if col.endswith('_expressed'):
            tissue = col.replace('_expressed', '')
            count = results_df[col].sum()
            total = len(results_df)
            percentage = (count / total * 100) if total > 0 else 0
            summary_stats.append({
                'Tissue': tissue,
                'Expressed_Count': count,
                'Total_Genes': total,
                'Percentage': f"{percentage:.1f}%"
            })

    if summary_stats:
        print("\n📊 Summary statistics by tissue:")
        summary_df = pd.DataFrame(summary_stats)
        display(summary_df)
else:
    print("❌ Results dataframe not found")


📊 Detailed tissue expression analysis results:

Results dataframe shape: (2207, 9)
Columns: ['Index', 'Gene_Symbol', 'Expressed_in_Any_Tissue', 'LAML_expressed', 'LAML_threshold', 'BRCA_expressed', 'BRCA_threshold', 'LUAD_expressed', 'LUAD_threshold']

📊 Complete tissue expression results:


,Index,Gene_Symbol,Expressed_in_Any_Tissue,LAML_expressed,LAML_threshold,BRCA_expressed,BRCA_threshold,LUAD_expressed,LUAD_threshold
0,0,ABCA10,False,False,5,False,3,False,4
1,1,ABCA4,False,False,5,False,3,False,4
2,2,ABCB11,False,False,5,False,3,False,4
3,3,ABCC11,False,False,5,False,3,False,4
4,4,ABCC3,True,False,5,False,3,True,4
...,...,...,...,...,...,...,...,...,...
2202,2202,ZNF827,True,False,5,True,3,False,4
2203,2203,ZNF99,False,False,5,False,3,False,4
2204,2204,ZPBP,False,False,5,False,3,False,4
2205,2205,ZSCAN21,True,False,5,True,3,False,4



📊 Summary statistics by tissue:


,Tissue,Expressed_Count,Total_Genes,Percentage
0,LAML,0,2207,0.0%
1,BRCA,981,2207,44.4%
2,LUAD,849,2207,38.5%
